### **[Imports]**

In [2]:
import requests
from bs4 import BeautifulSoup 
import csv
from datetime import datetime

# 전체 크롤링
import os
import time
import random

# postgre db 
# import psycopg2     

In [3]:
response = requests.get("https://www.spnews.co.kr/news/articleView.html?idxno=101229")  
soup = BeautifulSoup(response.text, 'html.parser') 
soup

<!DOCTYPE html>

<html class="no-js" data-mobile="false" dir="ltr" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=1080" name="viewport"/>
<meta content="SPN 서울평양뉴스" name="apple-mobile-web-app-title">
<meta content="max-image-preview:large" name="robots"/>
<!-- 메타태그정보 //-->
<link as="image" href="https://cdn.spnews.co.kr/news/photo/202511/101229_101943_843.jpg" rel="preload"/>
<meta content="[북한 경제 단신] 함남 북청군 과일가공공장 개건현대화공사 준공 - SPN 서울평양뉴스" name="title">
<meta content='북한 과수의 고향인 함경남도 북청군에서 과일가공공장 개건현대화공사를 마무리했다.노동신문은 12일 "수만㎡의 부지에 연건축면적이 1만 수천㎡에 이르는 사무청사와 생' name="description">
<meta content="북한N &gt; 경제/산업" name="Classification">
<meta content="SPN 서울평양뉴스" name="Copyright"/>
<meta content="안윤석 대기자" name="author"/>
<meta content="no-referrer-when-downgrade" name="referrer"/>
<meta content="SPN 서울평양뉴스" property="og:site_name"/>
<meta content="https://cdn.spnews.co.kr/news/photo/202511/101229_101943_843.jpg" property=

### **[Notes]**

In [4]:
soup.find("meta", property="og:url")

<meta content="https://www.spnews.co.kr/news/articleView.html?idxno=101229" property="og:url"/>

In [5]:
soup.html.body


<body class="" style="">
<nav class="skip-nav">
<a class="skip-nav-item" href="#user-menu" onclick="document.getElementById('user-menu').tabIndex = -1;document.getElementById('user-menu').focus();return false;">주요서비스 바로가기</a>
<a class="skip-nav-item" href="#user-container" onclick="document.getElementById('user-container').tabIndex = -1;document.getElementById('user-container').focus();return false;">본문 바로가기</a>
<a class="skip-nav-item" href="#user-footer" onclick="document.getElementById('user-footer').tabIndex = -1;document.getElementById('user-footer').focus();return false;">매체정보 바로가기</a>
<a class="skip-nav-item" href="#userLogin" onclick="document.getElementById('userLogin').tabIndex = -1;document.getElementById('userLogin').focus();return false;">로그인 바로가기</a>
<a class="skip-nav-item" href="#userSearch" onclick="document.getElementById('userSearch').tabIndex = -1;document.getElementById('userSearch').focus();return false;">기사검색 바로가기</a>
</nav>
<div class="off-canvas-content" data-o

<p style="text-align: justify;">

In [6]:
article_body = soup.find("div", class_ = "")
article_body

**[URL]**

In [7]:
url = soup.find("meta", property = "og:url")
article_url = url["content"]
article_url

'https://www.spnews.co.kr/news/articleView.html?idxno=101229'

**[Title]**

In [8]:
article_header = soup.find("h1", class_ = "heading")
article_header

<h1 class="heading">[북한 경제 단신] 함남 북청군 과일가공공장 개건현대화공사 준공</h1>

**[Author]**

In [9]:
tag = soup.find("i", class_="icon-user-o")
author = tag.parent.get_text(strip=True)  # '입력 2025.11.12 07:00'
author_clean = author.replace("기자명", "").strip()
author_clean

'안윤석 대기자'

**[Category]**

<a href="https://www.spnews.co.kr/news/articleList.html?sc_sub_section_code=S2N19&amp;view_type=sm">군사</a>
<a href="https://www.spnews.co.kr/news/articleList.html?sc_sub_section_code=S2N20&amp;view_type=sm">경제/산업</a>

In [10]:
a = soup.find("a", href = "https://www.spnews.co.kr/news/articleList.html?sc_sub_section_code=S2N20&amp;view_type=sm")
a

In [11]:
breadcrumb = soup.find("ul", class_="breadcrumbs")
category = None

for a in breadcrumb.find_all("a"):
    text = a.get_text(strip=True)
    if text in ["정치", "외교", "군사", "경제/산업", "사회/문화/체육"]:
        category = text
        break

print(category)

경제/산업


**[Publish_date]**

In [12]:
tag = soup.find("i", class_="icon-clock-o")
publish_date = tag.parent.get_text(strip=True)  # '입력 2025.11.12 07:00'
publish_date_clean = publish_date.replace("입력", "").strip()
publish_date_clean

'2025.11.12 07:00'

**[Contents]**

In [13]:
article_body = soup.find_all("span", style = "font-size:18px;")
article_body

[<span style="font-size:18px;">북한 과수의 고향인 함경남도 북청군에서 과일가공공장 개건현대화공사를 마무리했다.</span>,
 <span style="font-size:18px;">노동신문은 12일  "수만㎡의 부지에 연건축면적이 1만 수천㎡에 이르는 사무청사와 생산건물들을 개건하고 모든 공정이 자동화, 흐름선화된 과일가공품생산기지를 건설했다"고 보도했다.</span>,
 <span style="font-size:18px;">"공장에서는 과일농축즙, 과일단물, 과일살즙음료, 말린과일, 과일가루, 과일단묵생산공정을 완공한데 이어 최근 과일통졸임, 과일단졸임, 과일발효식초생산공정을 간춰 총 9가지에 달하는 과일가공품을 생산할 수 있다"고 소개했다.</span>,
 <span style="font-size:18px;">신문은 또 "황해북도농업과학연구소에서 지역의 기상기후적 특성에 맞는 두벌농사다수확재배기술을 개발도입했다"고 전했다.</span>,
 <span style="font-size:18px;">"연구소에서 올해 생육기일이 짧으면서도 가뭄견딜성이 강하고 불리한 토양조건에서도 수확고가 높은 다수확품종을 육종해 국가품종으로 등록되게 했으며, 재배기술을 확립했다"고 밝혔다.</span>,
 <span style="font-size:18px;">"연구소에서는 새로 육종한 다수확품종을 도내 농촌에 확대도입하고 사리원시와 서흥군, 은파군 등 여러 지역에 연구사들을 파견해 재배기술을 보급일반화하는 사업을 진행했다"고 신문은 덧붙였다.@ </span>]

In [14]:
contents = "\n".join(i.get_text(strip = True) for i in article_body)
contents_clean = contents.rstrip("@")
contents_clean

'북한 과수의 고향인 함경남도 북청군에서 과일가공공장 개건현대화공사를 마무리했다.\n노동신문은 12일\xa0\xa0"수만㎡의 부지에 연건축면적이 1만 수천㎡에 이르는 사무청사와 생산건물들을 개건하고 모든 공정이 자동화, 흐름선화된 과일가공품생산기지를 건설했다"고 보도했다.\n"공장에서는\xa0과일농축즙, 과일단물, 과일살즙음료, 말린과일, 과일가루, 과일단묵생산공정을 완공한데 이어 최근 과일통졸임, 과일단졸임, 과일발효식초생산공정을 간춰 총 9가지에 달하는 과일가공품을 생산할 수 있다"고 소개했다.\n신문은 또 "황해북도농업과학연구소에서 지역의 기상기후적 특성에 맞는 두벌농사다수확재배기술을 개발도입했다"고 전했다.\n"연구소에서 올해 생육기일이 짧으면서도 가뭄견딜성이 강하고 불리한 토양조건에서도 수확고가 높은 다수확품종을 육종해 국가품종으로 등록되게 했으며, 재배기술을 확립했다"고 밝혔다.\n"연구소에서는 새로 육종한 다수확품종을 도내 농촌에 확대도입하고 사리원시와 서흥군, 은파군 등 여러 지역에 연구사들을 파견해 재배기술을 보급일반화하는 사업을 진행했다"고 신문은 덧붙였다.'

### **[Crawler Ver1]**

In [15]:
class ArticleCrawling:

    def __init__(self, url):
        self.url = url
        self.soup = self._get_soup()

    def _get_soup(self):
        response = requests.get(self.url)
        return BeautifulSoup(response.text, 'html.parser')
    
    def get_article_url(self):
        meta_tag = self.soup.find("meta", property = "og:url")
        return meta_tag["content"] if meta_tag else None
    
    def get_id(self):
        url = self.get_article_url()    # https://www.spnews.co.kr/news/articleView.html?idxno=101049

        # source(spnew, ytn 등등)
        source = url.split("//")[1]
        source = source.split("/")[0]
        source = source.replace("www.", "").split(".")[0].lower()

        # id
        if "idxno" not in url:
            return None
        idx = url.split("idxno=")[1]
        return f"{source}_{idx}"
    
    def get_source(self):
        url = self.get_article_url()
        if not url:
            return None

        source = url.split("//")[1]
        source = source.split("/")[0]
        source = source.replace("www.", "").split(".")[0].lower()
        return source
    
    def get_title(self):
        header = self.soup.find("h1", class_ = "heading")
        return header.get_text(strip = True) if header else None
      
    def get_author(self):
        tag = self.soup.find("i", class_ = "icon-user-o")
        if tag:
            author = tag.parent.get_text(strip = True)
            return author.replace("기자명", "").strip()
        
    def get_section(self):
        breadcrumb = self.soup.find("ul", class_="breadcrumbs")
        if breadcrumb:
            for a in breadcrumb.find_all("a"):
                text = a.get_text(strip = True)
                if text in ["북한N", "서울&", "한반도W", "이슈+"]:   
                    return text
        return None
              
    def get_category(self):
        breadcrumb = self.soup.find("ul", class_ = "breadcrumbs")
        if breadcrumb:
            for a in breadcrumb.find_all("a"):
                text = a.get_text(strip = True)
                if text in ["정치", "외교", "군사", "경제/산업", "사회/문화/체육"]:
                    return text
    
    def get_publish_date(self):
        tag = self.soup.find("i", class_ = "icon-clock-o")
        if tag:
            raw = tag.parent.get_text(strip = True)
            raw = raw.replace("입력", "").strip()
            date = datetime.strptime(raw, "%Y.%m.%d %H:%M")  # 2025.11.17 07:47
            return date.strftime("%Y-%m-%d")
        
    def get_contents(self):
        article_body = self.soup.find_all("span", style = "font-size:18px;")
        contents = "\n".join(i.get_text(strip = True) for i in article_body)
        # return contents.rstrip("@") 
        return contents.split('@')[0].strip()
    
    def to_dict(self):
        return {
        
        "id": self.get_id(),
        "title": self.get_title(),
        "contents": self.get_contents(),
        "source": self.get_source(),
        "section": self.get_section(),
        "author": self.get_author(),
        "publish_date": self.get_publish_date(),
        "url": self.get_article_url(),
        "category": self.get_category(),

        
            # {
            #     "summary": ,
            #     "vector": ,
            #     "keywords": ,     >> 별개의 db로?
            # }
        }
    
    def __str__(self):
        data = self.to_dict()
        return "\n".join(f"{k}: {v}" for k, v in data.items())


### **[Crawler Ver2]**

In [16]:
class ArticleCrawling2:

    def __init__(self, url):
        self.url = url
        self.soup = self._get_soup()

    def _get_soup(self):
        response = requests.get(self.url)
        return BeautifulSoup(response.text, 'html.parser')
    
    def get_article_url(self):
        meta_tag = self.soup.find("meta", property = "og:url")
        return meta_tag["content"] if meta_tag else None
    
    def get_id(self):
        url = self.get_article_url()    # https://www.spnews.co.kr/news/articleView.html?idxno=101049

        # source(spnew, ytn 등등)
        source = url.split("//")[1]
        source = source.split("/")[0]
        source = source.replace("www.", "").split(".")[0].lower()

        # id
        if "idxno" not in url:
            return None
        idx = url.split("idxno=")[1]
        return f"{source}_{idx}"
    
    def get_source(self):
        url = self.get_article_url()
        if not url:
            return None

        source = url.split("//")[1]
        source = source.split("/")[0]
        source = source.replace("www.", "").split(".")[0].lower()
        return source
    
    def get_title(self):
        header = self.soup.find("h1", class_ = "heading")
        return header.get_text(strip = True) if header else None
      
    def get_author(self):
        tag = self.soup.find("i", class_ = "icon-user-o")
        if tag:
            author = tag.parent.get_text(strip = True)
            return author.replace("기자명", "").strip()
        
    def get_section(self):
        breadcrumb = self.soup.find("ul", class_="breadcrumbs")
        if breadcrumb:
            for a in breadcrumb.find_all("a"):
                text = a.get_text(strip = True)
                if text in ["북한N", "서울&", "한반도W", "이슈+"]:   
                    return text
        return None
              
    def get_category(self):
        breadcrumb = self.soup.find("ul", class_ = "breadcrumbs")
        if breadcrumb:
            for a in breadcrumb.find_all("a"):
                text = a.get_text(strip = True)
                if text in ["정치", "외교", "군사", "경제/산업", "사회/문화/체육"]:
                    return text
    
    def get_publish_date(self):
        tag = self.soup.find("i", class_ = "icon-clock-o")
        if tag:
            raw = tag.parent.get_text(strip = True)
            raw = raw.replace("입력", "").strip()
            date = datetime.strptime(raw, "%Y.%m.%d %H:%M")  # 2025.11.17 07:47
            return date.strftime("%Y-%m-%d")
        
    def get_contents(self):
        spans = self.soup.find_all("span", style = "font-size:18px;")
        if spans:
            contents = "\n".join(i.get_text(strip = True) for i in spans)
            return contents.split('@')[0].strip()
        
        articles = self.soup.find("article", id = "article-view-content-div")
        if articles:
            ps = articles.find_all("p")
            if ps:
                contents = "\n".join(p.get_text(strip=True) for p in ps)
                return contents.split("@")[0].strip()
                
        justify_ps = self.soup.find_all("p", style=lambda v: v and "text-align: justify" in v)
        if justify_ps:
            contents = "\n".join(p.get_text(strip=True) for p in justify_ps)
            return contents.split("@")[0].strip()
    
        return None
            
    def to_dict(self):
        return {
        
        "id": self.get_id(),
        "title": self.get_title(),
        "contents": self.get_contents(),
        "source": self.get_source(),
        "section": self.get_section(),
        "author": self.get_author(),
        "publish_date": self.get_publish_date(),
        "url": self.get_article_url(),
        "category": self.get_category(),

        
            # {
            #     "summary": ,
            #     "vector": ,
            #     "keywords": ,     >> 별개의 db로?
            # }
        }
    
    def __str__(self):
        data = self.to_dict()
        return "\n".join(f"{k}: {v}" for k, v in data.items())


### **[Crawler Test]**

In [17]:
test1 = ArticleCrawling("https://www.spnews.co.kr/news/articleView.html?idxno=101049")

In [18]:
print(test1)

id: spnews_101049
title: 北, 김영남 상임위원장 영결식...김정은 참석
contents: 고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로 치뤄졌다.
노동신문은 6일 김영남 전 상임위원장 장례소식을 전하면서 "고인의 영구가 안치된 서장회관은 특출한 공적을 남긴 노세대혁명가를 잃은 커다란 비애에 휩싸여있었다"고 보도했다.
"국가장의식에는 당 정치국 상무위원과 국가장의위원회 위원, 당중앙위원회와 최고인민회의 상임위원회, 내각, 성, 중앙기관, 무력기관의 일꾼, 고인의 유가족 등이 참석했다"고 전했다.
"추도곡이 울리는 가운데 고 김영남 상임위원장의 영구를 발인하는 의식이 진행됐으며, 수많은 평양시민들이 신미리애국열사릉으로 향한 연도마다에서 영구차에 조의를 표시했다"고 덧붙였다.
영결식은 신미리애국렬사릉에서 김정은 총비서가 참석한 가운데 열렸다.
"김 총비서는 당과 정부의 지도간부들과 열사릉 입구에서 고 김영남 동지의 영구를 맞이했다"고 신문은 전했다.
박태성 내각총리는 애도사에서 "김 상임위원장은 80여 성상 당을 따르고 당과 함께 영광의 일대기를 새겨오면서 국가의 정치사상적 위력과 권익을 수호하는데 뚜렷한 자욱을 새겨온 조선인민이 낳은 우수하고 열렬한 혁명가, 참다운 애국자의 한사람이었다"고 추모했다.
"김 총비서는 애국열사릉의 추모비 앞에서 전체 열사들에게 경의를 표시했다"고 신문은 덧붙였다.
source: spnews
section: 북한N
author: 안윤석 대기자
publish_date: 2025-11-06
url: https://www.spnews.co.kr/news/articleView.html?idxno=101049
category: 정치


In [19]:
test1_2 = ArticleCrawling2("https://www.spnews.co.kr/news/articleView.html?idxno=101049")

In [20]:
print(test1_2)

id: spnews_101049
title: 北, 김영남 상임위원장 영결식...김정은 참석
contents: 고(故) 김영남 북한 최고인민회의 상임위원회 위원장 장례식이 5일 평양에서 국장으로 치뤄졌다.
노동신문은 6일 김영남 전 상임위원장 장례소식을 전하면서 "고인의 영구가 안치된 서장회관은 특출한 공적을 남긴 노세대혁명가를 잃은 커다란 비애에 휩싸여있었다"고 보도했다.
"국가장의식에는 당 정치국 상무위원과 국가장의위원회 위원, 당중앙위원회와 최고인민회의 상임위원회, 내각, 성, 중앙기관, 무력기관의 일꾼, 고인의 유가족 등이 참석했다"고 전했다.
"추도곡이 울리는 가운데 고 김영남 상임위원장의 영구를 발인하는 의식이 진행됐으며, 수많은 평양시민들이 신미리애국열사릉으로 향한 연도마다에서 영구차에 조의를 표시했다"고 덧붙였다.
영결식은 신미리애국렬사릉에서 김정은 총비서가 참석한 가운데 열렸다.
"김 총비서는 당과 정부의 지도간부들과 열사릉 입구에서 고 김영남 동지의 영구를 맞이했다"고 신문은 전했다.
박태성 내각총리는 애도사에서 "김 상임위원장은 80여 성상 당을 따르고 당과 함께 영광의 일대기를 새겨오면서 국가의 정치사상적 위력과 권익을 수호하는데 뚜렷한 자욱을 새겨온 조선인민이 낳은 우수하고 열렬한 혁명가, 참다운 애국자의 한사람이었다"고 추모했다.
"김 총비서는 애국열사릉의 추모비 앞에서 전체 열사들에게 경의를 표시했다"고 신문은 덧붙였다.
source: spnews
section: 북한N
author: 안윤석 대기자
publish_date: 2025-11-06
url: https://www.spnews.co.kr/news/articleView.html?idxno=101049
category: 정치


In [21]:
test2 = ArticleCrawling("https://www.spnews.co.kr/news/articleView.html?idxno=95970")

In [22]:
print(test2)

id: spnews_95970
title: 北, 이재명 대통령 당선 첫 보도...노동신문 통해 주민들에게 알려
contents: 
source: spnews
section: 북한N
author: 안윤석 대기자
publish_date: 2025-06-05
url: https://www.spnews.co.kr/news/articleView.html?idxno=95970
category: 정치


In [23]:
test2_2 = ArticleCrawling2("https://www.spnews.co.kr/news/articleView.html?idxno=95970")

In [24]:
print(test2_2)

id: spnews_95970
title: 北, 이재명 대통령 당선 첫 보도...노동신문 통해 주민들에게 알려
contents: 북한이 이재명 대통령이 제21대 대통령으로 당선된 사실을 주민들에게 알렸다.
노동신문은 5일 "한국에서 지난해 '12.3비상계엄사태'로 대통령이 탄핵된 후 두 달 만인 6월 3일 대통령 선거가 진행됐다"며 이같이 보도했다.
이어 "선거에서는 더불어민주당 후보 리재명이 21대 대통령으로 당선됐다"고 덧붙였다.
신문은 6면에 관련 소식을 이렇게만 전하면서 특별한 논평은 하지 않았다.
북한이 21대 대통령선거에 대해 언급한 것은 이번이 처음이다.
북한은 2022년 제20대 대선 때는 선거 이틀 만에 "보수야당인 국민의힘의 후보 윤석열이 당선됐다"고 전했다.
source: spnews
section: 북한N
author: 안윤석 대기자
publish_date: 2025-06-05
url: https://www.spnews.co.kr/news/articleView.html?idxno=95970
category: 정치


In [25]:
test3 = ArticleCrawling("https://www.spnews.co.kr/news/articleView.html?idxno=94372")

In [26]:
print(test3)

id: spnews_94372
title: [오늘의 북한 날씨] 전 지역 가끔 구름많음
contents: 
source: spnews
section: 북한N
author: 안윤석 대기자
publish_date: 2025-04-16
url: https://www.spnews.co.kr/news/articleView.html?idxno=94372
category: 사회/문화/체육


In [27]:
test3_2 = ArticleCrawling2("https://www.spnews.co.kr/news/articleView.html?idxno=94372")

In [28]:
print(test3_2)

id: spnews_94372
title: [오늘의 북한 날씨] 전 지역 가끔 구름많음
contents: [기상청 발표 4월 16일(수) 북한 날씨]
□ 전 지역 가끔 구름많음.
□ 아침최저 평양 5도, 양강도 삼지연 -2도 예상.
source: spnews
section: 북한N
author: 안윤석 대기자
publish_date: 2025-04-16
url: https://www.spnews.co.kr/news/articleView.html?idxno=94372
category: 사회/문화/체육


### **[Auto Crawler]**

In [29]:
base_url = "https://www.spnews.co.kr"

In [30]:
BASE_URL = "https://www.spnews.co.kr/news/articleView.html?idxno={}"

class SPNCrawler:
    def __init__(self, start:101404, end, filename):
        self.start = start
        self.end = end
        self.filename = filename
        self.results = []

    def crawling_range(self):

        for i in range(self.start, self.end -1, -1):
            url = BASE_URL.format(i)

            try:
                crawler = ArticleCrawling(url)
                data = crawler.to_dict()

                if data.get("section") != "북한N":
                    continue
                
                self.results.append(data)

            except Exception as e:
                print(f"idx {i} 에서 에러 발생: {e}")
        
        return self.results

    def save_csv(self):
        data = self.results

        if not data:
            print("저장 데이터 없음")
            return
        
        keys = data[0].keys()

        with open(self.filename, "w", newline = "", encoding = "utf-8-sig") as f:
            writer = csv.DictWriter(f, fieldnames = keys)
            writer.writeheader()
            writer.writerows(data)

        print(f"{self.filename} 저장 완료.")

    def run(self):
        self.crawling_range()
        self.save_csv()
        return self.results

In [31]:
BASE_URL = "https://www.spnews.co.kr/news/articleView.html?idxno={}"

class SPNAllCrawler:
    def __init__(self, start, end, filename, section, sleep_min=0.5, sleep_max=1.0):
        self.start = start
        self.end = end
        self.filename = filename
        self.section = section
        self.sleep_min = sleep_min
        self.sleep_max = sleep_max
        self.existing_ids = self._load_existing_ids()

    def _load_existing_ids(self):
        ids = set()
        if os.path.exists(self.filename):
            with open(self.filename, "r", encoding="utf-8-sig", newline="") as f:
                reader = csv.DictReader(f)
                for row in reader:
                    if "id" in row and row["id"]:
                        ids.add(row["id"])
        return ids

    def _open_writer(self):
        file_exists = os.path.exists(self.filename)
        f = open(self.filename, "a", encoding="utf-8-sig", newline="")
        fieldnames = [
            "id",
            "title",
            "contents",
            "source",
            "section",
            "author",
            "publish_date",
            "url",       
            "category",            
        ]
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        return f, writer

    def run(self):
        f, writer = self._open_writer()

        try:
            for i in range(self.start, self.end - 1, -1):
                url = BASE_URL.format(i)

                try:
                    crawler = ArticleCrawling2(url)
                    data = crawler.to_dict()

                    # id 없으면 스킵
                    article_id = data.get("id")
                    if not article_id:
                        print(f"[SKIP] idx {i}: id 없음")
                        continue

                    # 이미 저장된 id면 스킵 (재실행 대비)
                    if article_id in self.existing_ids:
                        print(f"[SKIP] idx {i}: 이미 저장된 기사 ({article_id})")
                        continue

                    if self.section and data.get("section") != self.section:
                        print(f"[SKIP] idx {i}: section={data.get('section')}")
                        continue

                    # CSV 추가
                    writer.writerow(data)
                    f.flush()  
                    self.existing_ids.add(article_id)

                    print(f"[SAVE] idx {i}: {article_id} 저장 완료")

                except Exception as e:
                    print(f"[ERROR] idx {i} 에러 발생: {e}")

                # 서버 부하/차단 방지를 위한 랜덤 슬립
                time.sleep(random.uniform(self.sleep_min, self.sleep_max))

        finally:
            f.close()

In [32]:
%%time

crawler = SPNAllCrawler(
    start = 101404,
    end = 100000,                        
    filename = "spnews_db_test.csv",
    section = "북한N",          
    sleep_min = 0.7,
    sleep_max = 1.5                   # 조금 여유롭게
)

crawler.run()

# CPU times: total: 7h 47min 41s
# Wall time: 1d 52min 17s

[SAVE] idx 101404: spnews_101404 저장 완료
[SAVE] idx 101403: spnews_101403 저장 완료
[SAVE] idx 101402: spnews_101402 저장 완료
[SKIP] idx 101401: section=이슈+
[SKIP] idx 101400: section=서울&
[SKIP] idx 101399: section=서울&
[SKIP] idx 101398: section=서울&
[ERROR] idx 101397 에러 발생: 'NoneType' object has no attribute 'split'
[SKIP] idx 101396: section=서울&
[SKIP] idx 101395: section=이슈+
[SAVE] idx 101394: spnews_101394 저장 완료
[ERROR] idx 101393 에러 발생: 'NoneType' object has no attribute 'split'
[SAVE] idx 101392: spnews_101392 저장 완료
[SAVE] idx 101391: spnews_101391 저장 완료
[SAVE] idx 101390: spnews_101390 저장 완료
[SAVE] idx 101389: spnews_101389 저장 완료
[SKIP] idx 101388: section=서울&
[ERROR] idx 101387 에러 발생: 'NoneType' object has no attribute 'split'
[SKIP] idx 101386: section=서울&
[ERROR] idx 101385 에러 발생: 'NoneType' object has no attribute 'split'
[SKIP] idx 101384: section=서울&
[SAVE] idx 101383: spnews_101383 저장 완료
[SKIP] idx 101382: section=서울&
[SKIP] idx 101381: section=서울&
[ERROR] idx 101380 에러 발생: 'NoneT

KeyboardInterrupt: 

**[최종 결과]**

Count: 19,377

CPU times: total: 7h 47min 41s
Wall time: 1d 52min 17s